In [6]:
#Apriori算法
#构建数据集
def loadDataSet():
    # 事务ID 购买商品
    data = {'001': '面包，黄油，尿布，啤酒', '002': '咖啡，糖，小甜饼，鲑鱼，啤酒',
            '003': '面包，黄油，咖啡，尿布，啤酒，鸡蛋', '004': '面包，黄油，鲑鱼，鸡',
            '005': '鸡蛋，面包，黄油', '006': '鲑鱼，尿布，啤酒',
            '007': '面包，茶，糖鸡蛋', '008': '咖啡，糖，鸡，鸡蛋',
            '009': '面包，尿布，啤酒，盐', '010': '茶，鸡蛋，小甜饼，尿布，啤酒'}
    data_set = []
    for key in data:
        item = data[key].split('，')
        data_set.append(item)
    return data_set

#扫描loadDataSet返回的事务集，将所有的单项放入C中，构造出单项集
#参数：dataset-List列表，事务集
#返回值：List-冻结后的单项集列表
def createC1(dataSet):
    C = []
    for transaction in dataSet:
        for item in transaction:
            if [ item ] not in C:
                C.append([item])
    C.sort()
    
#调用frozenset函数将单项集列表冻结,返回冻结单项集列表
    return list(map(frozenset, C))
#根据minSupport参数设定的最小支持度，计算Ck的项集在原始记录D中的支持度，
#返回满足最小支持度的项集集合，以及所有项集支持度的字典
#参数：D-List列表，事务集
#     Ck-List列表，候选项集
#     minSupport-浮点数，最小支持度阈值
#返回值：retList-List列表，满足最小支持度要求的项集
#       supportData-Dict字典，项集对应的支持度
def scanD(D, Ck, minSupport):
    ssCnt = {}
# 对于每一个候选项集can，检查是否是D的一部分
# 即该候选can是否得到transaction的支持
    for tid in D:
        for can in Ck:     
            if can.issubset(tid):
                ssCnt[can] = ssCnt.get(can, 0) + 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
#每个项集的支持度
        support = ssCnt[key] / numItems       
#将满足最小支持度的项集，加入retList
        if support >= minSupport:
            retList.insert(0, key)            
#汇总支持度数据
        supportData[key] = round(support,6)
    return retList, supportData

#由初始候选项集的集合Ck生成新的候选项集，
#参数：Ck-List列表，k-1项集
#      k-整数，要生成的候选项集中的每项包含的元素个数
#返回值：retList-List列表，新生成的候选k项集
def aprioriGen(Ck, k):
    retList = []
    lenCk = len(Ck)
#以下的Ck[i]与Ck[j]均包含k-1个项，如果它们的前k-2项相同，
#则通过或运算将其拼接为一个包含k项的列表元素，加入到retList中
 # for i in range( lenLk ):
    #     for j in range( i + 1, lenLk ):
    #         L1 = list( Lk[ i ] )[ : k - 2 ];
    #         L2 = list( Lk[ j ] )[ : k - 2 ];
    #         L1.sort();L2.sort()
    #         if L1 == L2:
    #             retList.append( Lk[ i ] | Lk[ j ] ) 
    for i in range( lenCk ):
        for j in range( i + 1, lenCk ):
            L1 = Ck[ i ] |Ck[ j ]
            if(len(L1)==k):
                if L1 not in retList:
                    retList.append( L1 ) 
    return retList

#根据minSupport参数设定的最小支持度，返回所有满足最小支持度的项集，
#参数：D-List列表，事务集
#     minSupport-浮点数，最小支持度阈值
#返回值：L-List列表，所有满足最小支持度要求的项集
#       supportData-Dict字典，项集对应的支持度
def apriori(D, minSupport):
    C1=createC1(D)
    L1, suppData = scanD(D, C1, minSupport)
    L = [L1]
#最初的L1中的每个项集含有一个元素，
#新生成的项集应该含有2个元素，所以 k=2
    k = 2    
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)        
#将新的项集的支持度数据加入原来的总支持度字典中
        suppData.update(supK)        
#将符合最小支持度要求的项集加入L
        L.append(Lk)        
#新生成的项集中的元素个数应不断增加
        k += 1
#返回所有满足条件的频繁项集的列表，和所有候选项集的支持度信息
    return L, suppData

#计算规则的置信度 
#参数：freqSet-frozenset不可变集合，频繁项集
#     H-不可变集合，频繁项集中所有的元素
#     supportData-dic字典，频繁项集中所有元素的支持度
#     brl-tuple元组，满足可信度条件的关联规则
#     minConf-浮点数，最小置信度阈值，默认值0.7
#返回值：prunedH-List列表，满足最小置信度要求的规则
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

#对频繁项集中元素超过2的项集进行合并
#参数：freqSet-frozenset不可变集合，频繁项集
#     H-不可变集合，频繁项集中所有的元素
#     supportData-dic字典，频繁项集中所有元素的支持度
#     brl-tuple元组，满足可信度条件的关联规则
#     minConf-浮点数，最小置信度阈值，默认值0.7
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])  
#检查频繁项集是否大到移除大小为 m的子集
    if len(freqSet) > m+1:
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
#如果不止一条规则满足要求，进一步递归合并
        if len(Hmp1) > 1:
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

#根据频繁项集和最小可信度生成规则
#参数：L-list,列表，频繁项集
#     supportData-dict字典，所有项集（不仅仅是频繁项集）的支持度西南西
#     minConf-浮点数，最小置信度阈值，默认值0.7
#返回值：bigRuleList-List列表，满足最小置信度要求的规则
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[ i ]:
#对于每一个频繁项集的集合freqSet
            H1 = [frozenset([item]) for item in freqSet]
#如果频繁项集中的元素个数大于2，需要进一步合并
            if i > 1:
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

if __name__ == '__main__':   
    # 导入数据集
    myDat = loadDataSet()    
# 选择频繁项集
# 选择出支持度不小于2项（共9笔交易，因此支持率为0.22） 的项集作为频繁项集
    L, suppData = apriori(myDat, 0.22)
    print("频繁项集L：", L,'\n')
    print("所有候选项集的支持度信息：", suppData,'\n')
    print("所有规则的置信度信息：")
    rules = generateRules(L,suppData,0.1)

频繁项集L： [[frozenset({'鸡蛋'}), frozenset({'鲑鱼'}), frozenset({'咖啡'}), frozenset({'黄油'}), frozenset({'面包'}), frozenset({'尿布'}), frozenset({'啤酒'})], [frozenset({'啤酒', '尿布'}), frozenset({'啤酒', '面包'}), frozenset({'面包', '尿布'}), frozenset({'黄油', '面包'})], [frozenset({'面包', '啤酒', '尿布'})], []] 

所有候选项集的支持度信息： {frozenset({'啤酒'}): 0.6, frozenset({'尿布'}): 0.5, frozenset({'面包'}): 0.6, frozenset({'黄油'}): 0.4, frozenset({'咖啡'}): 0.3, frozenset({'小甜饼'}): 0.2, frozenset({'糖'}): 0.2, frozenset({'鲑鱼'}): 0.3, frozenset({'鸡蛋'}): 0.4, frozenset({'鸡'}): 0.2, frozenset({'糖鸡蛋'}): 0.1, frozenset({'茶'}): 0.2, frozenset({'盐'}): 0.1, frozenset({'黄油', '面包'}): 0.4, frozenset({'黄油', '尿布'}): 0.2, frozenset({'黄油', '啤酒'}): 0.2, frozenset({'面包', '尿布'}): 0.3, frozenset({'啤酒', '面包'}): 0.3, frozenset({'啤酒', '尿布'}): 0.5, frozenset({'鲑鱼', '咖啡'}): 0.1, frozenset({'鲑鱼', '啤酒'}): 0.2, frozenset({'啤酒', '咖啡'}): 0.2, frozenset({'咖啡', '鸡蛋'}): 0.2, frozenset({'黄油', '鸡蛋'}): 0.2, frozenset({'面包', '鸡蛋'}): 0.2, frozenset({'鸡蛋', '尿布'}): 0.2, f

In [4]:
from collections import defaultdict
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import train_test_split
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n
# First train an UCF algorithm on the movielens dataset.
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25)#测试集比例占25%
algo = KNNBasic(k=50, min_k=10,sim_options ={'name':'cosine','user_based': True}, verbose=True) #建立UCF算法并训练
algo.fit(trainset)
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
predictions = algo.test(testset)

#Evaluate Metrics 评价指标
accuracy.rmse(predictions)# Then compute RMSE
accuracy.mae(predictions)#Then compute NAE
n=5 #推荐列表长度
top_n = get_top_n(predictions, n)
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print('UserID:',uid, 'Top-'+str(n)+' Recommender List',[iid for (iid, _) in user_ratings])

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0185
MAE:  0.8040
UserID: 479 Top-5 Recommender List ['483', '89', '50', '511', '174']
UserID: 299 Top-5 Recommender List ['483', '127', '474', '136', '89']
UserID: 104 Top-5 Recommender List ['50', '9', '246', '126', '269']
UserID: 896 Top-5 Recommender List ['12', '511', '480', '134', '515']
UserID: 890 Top-5 Recommender List ['127', '134', '50', '515', '173']
UserID: 870 Top-5 Recommender List ['56', '12', '172', '191', '513']
UserID: 658 Top-5 Recommender List ['318', '127', '22', '488', '96']
UserID: 11 Top-5 Recommender List ['12', '427', '603', '79', '190']
UserID: 440 Top-5 Recommender List ['313', '515', '1194', '198', '923']
UserID: 711 Top-5 Recommender List ['169', '315', '213', '50', '98']
UserID: 524 Top-5 Recommender List ['483', '603', '64', '654', '134']
UserID: 378 Top-5 Recommender List ['174', '197', '172', '132', '22']
UserID: 387 Top-5 Recommender List ['12', '127', '318', '23', '1

UserID: 789 Top-5 Recommender List ['475', '276', '1012', '1161', '628']
UserID: 131 Top-5 Recommender List ['242', '137', '126', '1', '297']
UserID: 728 Top-5 Recommender List ['282', '286', '237', '871']
UserID: 96 Top-5 Recommender List ['483', '50', '98', '194', '100']
UserID: 921 Top-5 Recommender List ['50', '194', '275', '96', '185']
UserID: 461 Top-5 Recommender List ['302', '305', '347', '327']
UserID: 475 Top-5 Recommender List ['316', '315', '286', '70', '303']
UserID: 611 Top-5 Recommender List ['750', '311', '347', '1243', '334']
UserID: 646 Top-5 Recommender List ['313', '750', '310', '328', '319']
UserID: 231 Top-5 Recommender List ['15', '126', '1', '405', '121']
UserID: 695 Top-5 Recommender List ['302', '887', '333', '270', '301']
UserID: 526 Top-5 Recommender List ['1007', '315', '750', '1', '346']
UserID: 651 Top-5 Recommender List ['242', '116', '690', '327', '683']
UserID: 8 Top-5 Recommender List ['187', '50', '96', '190', '174']
UserID: 428 Top-5 Recommender Lis